<a href="https://colab.research.google.com/github/GuilhermeCaly/fraud_credit/blob/main/fraud_credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
import pandas as pd
import numpy as np
from datetime import datetime
import pymysql
from google.colab import userdata

In [9]:
host = userdata.get('host')
user = userdata.get('user')
password = userdata.get('password')
database = userdata.get('database')
drive.mount('/content/drive')
file_path = userdata.get('file_path')
dt_source_lc = pd.read_csv(file_path)
df_fraud =  dt_source_lc

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
df_fraud.dropna(inplace=True)
df_fraud = df_fraud[(df_fraud['location_region'] != '0') & (df_fraud['amount'] != "none") & (df_fraud['risk_score'] != "none")]
df_fraud = df_fraud[(df_fraud['amount'] != "none") & (df_fraud['risk_score'] != "none")]
df_fraud['timestamp'] = pd.to_datetime(df_fraud['timestamp'], unit='s')

In [11]:
df_fraud['sending_address'] =  df_fraud['sending_address'].astype(str)
df_fraud['risk_score'] =  df_fraud['risk_score'].astype(float)
df_fraud['location_region'] =  df_fraud['location_region'].astype(str)
df_fraud['receiving_address'] =  df_fraud['receiving_address'].astype(str)
df_fraud['amount'] =  df_fraud['amount'].astype(float)
df_fraud['transaction_type'] =  df_fraud['transaction_type'].astype(str)
df_fraud['ip_prefix'] =  df_fraud['ip_prefix'].astype(str)
df_fraud['login_frequency'] =  df_fraud['login_frequency'].astype(int)
df_fraud['session_duration'] =  df_fraud['session_duration'].astype(int)
df_fraud['purchase_pattern'] =  df_fraud['purchase_pattern'].astype(str)
df_fraud['age_group'] =  df_fraud['age_group'].astype(str)
df_fraud['anomaly'] =  df_fraud['anomaly'].astype(str)

In [ ]:
mean_risk_score = df_fraud.groupby('location_region')['risk_score'].mean()
result_table_1 = mean_risk_score.sort_values(ascending=False).reset_index()
display(result_table_1)

In [ ]:
sales_data = df_fraud[df_fraud['transaction_type'] == 'sale']
latest_transactions = sales_data.loc[sales_data.groupby('receiving_address')['timestamp'].idxmax()]
result_table_2 = latest_transactions.nlargest(3, 'amount')[['receiving_address', 'amount', 'timestamp']]
result_table_2

In [40]:
log_error = dt_source_lc.copy()
log_error = dt_source_lc[(dt_source_lc['location_region'] == '0') |
                         (dt_source_lc['amount'] == "none") |
                         (dt_source_lc['risk_score'] == "none")]

In [ ]:
log_error['error_message'] = ''
log_error.loc[log_error['location_region'] == '0', 'error_message'] = 'REGION DOES NOT EXISTS'
log_error.loc[log_error['amount'] == "none", 'error_message'] = 'AMOUNT IS NONE'
log_error.loc[log_error['risk_score'] == "none", 'error_message'] = 'RISK SCORE IS NONE'

In [16]:
quantidade_erros_log = len(log_error)
erros_region = log_error.loc[log_error['error_message'] == 'REGION DOES NOT EXISTS']
erros_region_all = len(erros_region)
erros_amount = log_error.loc[log_error['error_message'] == 'AMOUNT IS NONE']
erros_amount = len(erros_amount)
erros_risk_score = log_error.loc[log_error['error_message'] == 'RISK SCORE IS NONE']
erros_risk_score = len(erros_risk_score)
quantidade_total_registros = len(dt_source_lc)
percentual_conformidade_log = 100 * (quantidade_total_registros - quantidade_erros_log) / quantidade_total_registros


In [17]:
last_insert_date = datetime.now()

data_quality= {
    'qtd_erros_Log': [quantidade_erros_log],
    'qtd_erros_region__not_exits': [erros_region_all],
    'qtd__erros_amount_none': [erros_amount],
    'qtd_erros_risk_none': [erros_risk_score],
    'qtd_total_registros': [quantidade_total_registros],
    'percent_conf_log': [percentual_conformidade_log],
    'Dt_ultimo_inset': [last_insert_date]
}
data_quality = pd.DataFrame(data_quality)

In [18]:
data_quality = pd.DataFrame(data_quality)

In [64]:
conn = pymysql.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

In [65]:
conn.open
cursor = conn.cursor()

In [26]:
for index, row in data_quality.iterrows():
    insert_tbl_dt_qlt = "INSERT INTO DB_FRAUD.DT_QTLT (qtd_erros_log, qtd_erros_region__not_exits, qtd__erros_amount_none, qtd_erros_risk_none, qtd_total_registros, percent_conf_log, Dt_ultimo_inset) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    data_quality_values = (row['qtd_erros_Log'], row['qtd_erros_region__not_exits'], row['qtd__erros_amount_none'], row['qtd_erros_risk_none'], row['qtd_total_registros'], row['percent_conf_log'], row['Dt_ultimo_inset'])
    cursor.execute(insert_tbl_dt_qlt, data_quality_values)
    conn.commit()

In [ ]:
for index , row in log_error.iterrows():
    insert_logerror = "INSERT INTO DB_FRAUD.DT_LOGERROR(`timestamp` ,`sending_address`  ,`receiving_address`    ,`amount`   ,`transaction_type` ,`location_region`  ,`ip_prefix`    ,`login_frequency`  ,`session_duration` ,`purchase_pattern` ,`age_group`    ,`risk_score`   ,`anomaly`  ,`error_message`)  VALUES (%s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s) "
    log_error_insert = (row['timestamp'],row['sending_address'],row['receiving_address'],row['amount'],row['transaction_type'],row['location_region'],row['ip_prefix'],row['login_frequency'],row['session_duration'],row['purchase_pattern'],row['age_group'],row['risk_score'],row['anomaly'],row['error_message'],)
    cursor.execute(insert_logerror, log_error_insert)
    conn.commit()

In [68]:
for index , row in result_table_1.iterrows():
    insert_result_table_1 = "INSERT INTO DB_FRAUD.RSLT_I(location_region,risk_score)  VALUES (%s, %s) "
    result_table_1_insert = (row['location_region'],row['risk_score'])
    cursor.execute(insert_result_table_1, result_table_1_insert)
    conn.commit()

In [61]:
for index , row in result_table_2.iterrows():
    insert_result_table_2 = "INSERT INTO DB_FRAUD.RSLT_2(receiving_address,amount,timestamp)  VALUES (%s, %s,%s) "
    result_table_2_insert = (row['receiving_address'],row['amount'], row['timestamp'])
    cursor.execute(insert_result_table_2, result_table_2_insert)
    conn.commit()

In [62]:
    cursor.close()
    conn.close()

Conexão ao MySQL fechada.
